In [7]:
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality, get_column_plot
from sdv.single_table import GaussianCopulaSynthesizer, CTGANSynthesizer, TVAESynthesizer, CopulaGANSynthesizer

import pandas as pd

### Load dataset

In [8]:
data = pd.read_csv('datasets/breast.csv')

data.head()

,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,fractal_dimension1,...,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3,Diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,M
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,M
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,M
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,M
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,M


In [9]:
metadata = SingleTableMetadata()

# Detect metadata from CSV file
metadata.detect_from_csv(filepath='datasets/breast.csv')

metadata

{
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "columns": {
        "radius1": {
            "sdtype": "numerical"
        },
        "texture1": {
            "sdtype": "numerical"
        },
        "perimeter1": {
            "sdtype": "numerical"
        },
        "area1": {
            "sdtype": "numerical"
        },
        "smoothness1": {
            "sdtype": "numerical"
        },
        "compactness1": {
            "sdtype": "numerical"
        },
        "concavity1": {
            "sdtype": "numerical"
        },
        "concave_points1": {
            "sdtype": "numerical"
        },
        "symmetry1": {
            "sdtype": "numerical"
        },
        "fractal_dimension1": {
            "sdtype": "numerical"
        },
        "radius2": {
            "sdtype": "numerical"
        },
        "texture2": {
            "sdtype": "numerical"
        },
        "perimeter2": {
            "sdtype": "numerical"
        },
        "area2": {
            "sdty

### GaussianCopulaSynthesizer

In [13]:
synthesizer = GaussianCopulaSynthesizer(
    metadata
)
synthesizer.fit(data)
synthesizer.save(filepath='synthesizers/GCSbc.pkl')
#synthesizer.get_learned_distributions()

In [ ]:
# or load it
synthesizer = GaussianCopulaSynthesizer.load(
    filepath='synthesizers/GCSbc.pkl'
)
synthetic_data = synthesizer.sample(
    num_rows=1000,
    batch_size=100,
    output_file_path='synthetic/GCSbc.csv',
)

### CTGANSynthesizer

In [ ]:
synthesizer = CTGANSynthesizer(
    metadata,
    epochs=500,
)
synthesizer.fit(data)
synthesizer.save(filepath='synthesizers/CTGANbc.pkl')
synthesizer.get_loss_values()

In [ ]:
# or load it
synthesizer = CTGANSynthesizer.load(
    filepath='synthesizers/CTGANbc.pkl'
)
synthetic_data = synthesizer.sample(
    num_rows=1000,
    batch_size=100,
    output_file_path='synthetic/CTGANbc.csv',
)

### TVAESynthesizer

In [31]:
synthesizer = TVAESynthesizer(
    metadata,
    epochs=500,
)
synthesizer.fit(data)
synthesizer.save(filepath='synthesizers/TVAESbc.pkl')

In [32]:
synthesizer.get_loss_values()

,Epoch,Batch,Loss
0,0,0,149.792953
1,0,1,124.313095
2,1,0,132.695435
3,1,1,114.668709
4,2,0,121.784889
...,...,...,...
995,497,1,-25.037146
996,498,0,-24.948425
997,498,1,-28.594090
998,499,0,-24.656824


In [ ]:
# or load it
synthesizer = TVAESynthesizer.load(
    filepath='synthesizers/TVAESbc.pkl'
)

In [33]:
synthetic_data = synthesizer.sample(
    num_rows=100_000,
    batch_size=100,
    output_file_path='synthetic/TVAESbc.csv',
)

Sampling rows: 100%|██████████| 100000/100000 [03:06<00:00, 536.03it/s]


### CopulaGANSynthesizer

In [34]:
# train the synthesizer 
synthesizer = CopulaGANSynthesizer(
    metadata,
    epochs=500,
    verbose=True,
)

synthesizer.fit(data)
synthesizer.save(filepath='synthesizers/CGANbc.pkl')

Gen. (-2.26) | Discrim. (-0.23): 100%|██████████| 500/500 [00:23<00:00, 21.68it/s]


In [35]:
synthesizer.get_loss_values()

,Epoch,Generator Loss,Discriminator Loss
0,0,0.817293,-0.003665
1,1,0.770676,-0.029333
2,2,0.782333,-0.093022
3,3,0.783737,-0.113403
4,4,0.739277,-0.188489
...,...,...,...
495,495,-2.712113,-0.443949
496,496,-2.386628,-0.423564
497,497,-2.203330,-0.059741
498,498,-2.323760,0.019763


In [ ]:
# or load it
synthesizer = CopulaGANSynthesizer.load(
    filepath='synthesizers/CGANbc.pkl'
)

In [36]:
synthetic_data = synthesizer.sample(
    num_rows=100_000,
    batch_size=100,
    output_file_path='synthetic/CGANbc.csv',
)

Sampling rows: 100%|██████████| 100000/100000 [04:25<00:00, 377.00it/s]
